In [1]:
%load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
from tqdm import tqdm
#from tqdm.tqdm_notebook import tqdm_notebook

In [2]:
df = pd.read_csv("clean_chemical_master_gwq_data_sample.csv")
df.head()

,well_id,results,chemical,date,units,qualifer,rl,latitude,longitude,well_type,source
0,ANT-01,0.003,"1,1,1-Trichloroethane",1/28/2008,UG/L,<,0.0030,34.624472,-118.185056,MUNICIPAL,USGS
1,ANT-01,0.005,"1,1,2,2-Tetrachloroethane",1/28/2008,UG/L,<,0.0050,34.624472,-118.185056,MUNICIPAL,USGS
2,ANT-01,0.006,"1,2-Dibromo-3-chloropropane",1/28/2008,UG/L,<,0.0059,34.624472,-118.185056,MUNICIPAL,USGS
3,ANT-01,0.006,"1,2-Dichlorobenzene",1/28/2008,UG/L,<,0.0060,34.624472,-118.185056,MUNICIPAL,USGS
4,ANT-01,0.006,Alachlor,1/28/2008,UG/L,<,0.0060,34.624472,-118.185056,MUNICIPAL,USGS


time: 67.8 ms


In [3]:
df['latitude'] = df['latitude'].astype(float)

time: 997 µs


In [4]:
df['longitude'] = df['longitude'].astype(float)

time: 997 µs


In [7]:
geolocator = Nominatim(user_agent="untitled.py")
location = geolocator.reverse("34.624472, -118.185056")
geo_string = location.address.split(",")
geo_string[4]

' Los Angeles County'

time: 655 ms


In [9]:
county_column[:10]

[' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County',
 ' Los Angeles County']

time: 1.99 ms


In [11]:
df["County"] = county_column
df.head(100)

,well_id,results,chemical,date,units,qualifer,rl,latitude,longitude,well_type,source,County
0,ANT-01,0.003,"1,1,1-Trichloroethane",1/28/2008,UG/L,<,0.0030,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
1,ANT-01,0.005,"1,1,2,2-Tetrachloroethane",1/28/2008,UG/L,<,0.0050,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
2,ANT-01,0.006,"1,2-Dibromo-3-chloropropane",1/28/2008,UG/L,<,0.0059,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
3,ANT-01,0.006,"1,2-Dichlorobenzene",1/28/2008,UG/L,<,0.0060,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
4,ANT-01,0.006,Alachlor,1/28/2008,UG/L,<,0.0060,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
...,...,...,...,...,...,...,...,...,...,...,...,...
75,ANT-01,7.500,Tetrachloroethene,1/28/2008,PH UNITS,NaN,NaN,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
76,ANT-01,7.300,Trichloroethene,1/28/2008,MG/L,NaN,NaN,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
77,ANT-01,-9.600,Trifluralin,1/28/2008,per mil,NaN,NaN,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County
78,ANT-01,1120.000,Vinyl chloride,1/28/2008,UMHOS/CM,NaN,NaN,34.624472,-118.185056,MUNICIPAL,USGS,Los Angeles County


time: 28.9 ms


In [8]:
county_column = []
for i in range(len(df)):
    # obtain lat and lon for each row in data frame
    lat = df.latitude.iloc[i]
    lon = df.longitude.iloc[i]
    
    # get the address,whatever using lat and long
    location = geolocator.reverse(f"{lat}, {lon}")
    geo_string = location.address.split(",")
    county_column.append(geo_string[4]) #get county

df2["County"] = county_column

# for row in df.iterrows():
#     geolocator = Nominatim(user_agent="untitled.py")
#     #location = geolocator.reverse("latitude, longitude")
#     location = geolocator.reverse([8], [9])
#     geo_string = location.address.split(",")
#     geo_string[2]
    
# df2 = geo_string[2]    
# df2.head()

NameError: name 'df2' is not defined

time: 48.3 s


In [ ]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1


In [30]:
df2.length


AttributeError: 'str' object has no attribute 'length'

time: 11.5 ms


In [ ]:
df["location"] = df["latitude"].map(str) + "," + df["longitude"].map(str)
df["location"][0]

In [ ]:
locator = Nominatim(user_agent="myGeocoder", timeout=30)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.1)

In [ ]:
tqdm.pandas()
df['county'] = df['location'].progress_apply(rgeocode)
df.head()